## Notebook for getting the expected test results for an MS2Deepscore model

This can be used to check that the expected results are given when running the model in MZMine. 
These results are used in the tests for mzmine found: https://github.com/mzmine/mzmine/blob/master/mzmine-community/src/test/java/io/github/mzmine/util/scans/similarity/impl/ms2deepscore/MS2DeepscoreModelTest.java

In [15]:
from math import isclose

import numpy as np
import torch
from matchms import Spectrum

from ms2deepscore import MS2DeepScore
from ms2deepscore.models import load_model

In [12]:
model_file_name = "../../../ms2deepscore/ms2deepscore/tests/resources/ms2deepscore_model.pt"

In [11]:
def test_siamese_model_forward_pass(model_file_name):
    model = load_model(model_file_name)
    similarity_score = model(torch.tensor([np.array([0.1]*990), np.array([0.2]*990)], dtype=torch.float32),
                             torch.tensor([np.array([0.2]*990), np.array([0.1]*990)], dtype=torch.float32),
                             torch.tensor([np.array([0.] * 2), np.array([1.] * 2)], dtype=torch.float32),
                             torch.tensor([np.array([1.] * 2), np.array([0.] * 2)], dtype=torch.float32))
    assert similarity_score.shape[0] == 2
    print(similarity_score)


def test_siamese_model_embedding_generation_from_tensor(model_file_name):
    """This test is to compare output of a test model with the output in MZMine for the same model"""
    model = load_model(model_file_name)
    similarity_score = model.encoder(torch.tensor([np.array([0.1]*990), np.array([0.2]*990)], dtype=torch.float32),
                  torch.tensor([np.array([0.] * 2), np.array([1.] * 2)], dtype=torch.float32),
                  )
    assert similarity_score.shape == (2, 50)
    assert isclose(float(similarity_score[0][0]), -4.6007e-02, abs_tol=0.001)
    assert isclose(float(similarity_score[1][0]), -3.7386e-02, abs_tol=0.001)


def test_siamese_model_embedding_generation_from_spectrum(model_file_name):
    model = load_model(model_file_name)
    ms2deepscore_model = MS2DeepScore(model)

    test_spectra = [Spectrum(mz=np.array([100.1, 200.1, 300.1, 400.1, 500.1]), intensities=np.array([0.2, 0.4, 0.6, 0.8, 1.0]),
                             metadata={"precursor_mz": 600,
                                       "ionmode": "positive"
                                       }),
                    Spectrum(mz=np.array([600.1, 700.1, 800.1, 900.1, 1000.1]), intensities=np.array([0.2, 0.4, 0.6, 0.8, 1.0]),
                             metadata={"precursor_mz": 1000,
                                       "ionmode": "positive"
                                       })]

    embeddings = ms2deepscore_model.get_embedding_array(test_spectra)
    print(embeddings)
    assert embeddings.shape == (2, 50)
    scores = ms2deepscore_model.matrix(test_spectra, test_spectra)
    print(scores)

In [14]:
test_siamese_model_forward_pass(model_file_name)
test_siamese_model_embedding_generation_from_tensor(model_file_name)
test_siamese_model_embedding_generation_from_spectrum(model_file_name)

tensor([0.9912, 0.9912], grad_fn=<SumBackward1>)


2it [00:00, ?it/s]


[[-0.06332766 -0.11024649 -0.08610675 -0.00525632 -0.079157   -0.08691251
   0.09449271  0.07916132  0.0580234  -0.02235537 -0.04483622 -0.0624437
  -0.0950591   0.01551783  0.04003913  0.05260848 -0.03278536 -0.07231505
  -0.0382287  -0.04389135 -0.00019184  0.09974964 -0.09984992  0.06247782
  -0.0609279   0.01529961  0.02764707 -0.03331405 -0.09643134 -0.01307281
  -0.04938746 -0.0353475  -0.11392142  0.09726321 -0.06136806 -0.07213654
  -0.01545145  0.01832605  0.08730272  0.06531193  0.09753364  0.04749898
   0.11662124  0.05686804 -0.01961342  0.03254474  0.0610045   0.01302657
   0.05929426  0.05524265]
 [-0.05749714 -0.09926251 -0.08300484 -0.00320202 -0.07668673 -0.07920866
   0.09071646  0.08084232  0.05746764 -0.01302737 -0.03829534 -0.06739624
  -0.09885679  0.01685361  0.03426528  0.04910641 -0.01993307 -0.07650086
  -0.03133395 -0.04873285  0.00392702  0.09327801 -0.0932356   0.06813178
  -0.05970918  0.01320491  0.03372256 -0.03812464 -0.09187092 -0.01001406
  -0.0464534

2it [00:00, 481.55it/s]
2it [00:00, 666.24it/s]


[[1.         0.99633499]
 [0.99633499 1.        ]]
